In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder

import os
from PIL import Image

import pandas as pd

In [2]:
path = 'C:\\Users\\JungHyeona\\Documents\\Project\\AI_deeplearning\\image\\test'

In [3]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1)
        self.conv1 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.fc1 = nn.Linear(32*32*3, 256)
        self.fc2 = nn.Linear(256, 2)
 
    def forward(self, x):
        
        x = self.conv1(x)
        x = self.pool(F.relu(x))
        x = self.conv2(x)
        x = self.pool(F.relu(x))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x
    
model = MyModel()

In [4]:
# 데이터 전처리
transform = transforms.Compose([
     transforms.Resize((64, 64)),
     transforms.ToTensor(),
     transforms.Normalize((0.111850675, 0.11192831, 0.11306531), (0.029490899, 0.0293483175, 0.029255329999999996)),
     transforms.Grayscale(num_output_channels=1)
    ])

In [5]:
images = []

for test in os.listdir(path):  # test = Block, Normal, Pattern, Scanline
    folder = os.path.join(path, test)  # folder = path + Block, Normal, Pattern, Scanline
    
    for file in os.listdir(folder):
        # img_folder = Block, Normal, Pattern, Scanline
        img_file = os.path.join(folder, file)    # img_folder = path/test/Block, Normal, Pattern, Scanline

        if img_file.endswith('.PNG'):
            image = Image.open(img_file)
            image = transform(image)
            image = image.unsqueeze(0)
            
            images.append(image)

In [8]:
len(images)

2357

In [9]:
save_checkpoint = torch.load('C:\\Users\\JungHyeona\\Documents\\Project\\AI_deeplearning\\image\\B_cnn01.pt')
model.load_state_dict(save_checkpoint, strict=False)
model.eval()

# https://paka96.tistory.com/46

MyModel(
  (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=3072, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=2, bias=True)
)

## 결과 값 dataframe으로 저장

In [10]:
df = pd.DataFrame(columns=['predict', 'file_name'])

In [11]:
df

,predict,file_name


In [12]:
# 예측
with torch.no_grad():
    outputs = model(image)

# 결과 확인
_, predicted = torch.max(outputs, 1)

# 결과 DataFrame에 추가
result = 't' if predicted.item() == 1 else 'f'
results_df = results_df.append({'Image': image_path, 'Result': result}, ignore_index=True)

RuntimeError: Given groups=1, weight of size [128, 64, 3, 3], expected input[1, 1, 64, 64] to have 64 channels, but got 1 channels instead